In [1]:
import pandas as pd
import numpy

In [2]:
import codecs
import os
with codecs.open('customer_data.csv', 'r') as file:
    BLOCKSIZE = 512
    if not os.path.exists('customer_data(utf-8).csv'):
        with codecs.open('customer_data(utf-8).csv', "w", "utf-8") as targetFile:
            while True:
                contents = file.read(BLOCKSIZE)
                if not contents:
                    break
                targetFile.write(contents)

FileNotFoundError: [Errno 2] No such file or directory: 'customer_data.csv'

In [2]:
dtype = {
    '交易id': numpy.str,
    '資料日期': numpy.str,
    '資料時間': numpy.str,
    '餐別帶': numpy.str,
    '縣市別': numpy.str,
    '店舖代號': numpy.uint32,
    '主商圈': numpy.str,
    '品號-品名稱': numpy.str,
    '群號-群名稱': numpy.str,
    '單品名稱': numpy.str,
    '銷售數量': numpy.uint16,
    '銷售單價': numpy.float,
    '交易金額': numpy.float
}
USE_COLUMNS = ['交易id', '資料日期', '資料時間', '餐別帶', '縣市別', '店舖代號', '主商圈', '品號-品名稱',
       '群號-群名稱', '單品名稱', '銷售數量', '銷售單價', '交易金額']
PARSE_DATES = {
    '資料日期與時間': [
        '資料日期',
        '資料時間'
    ]
}

In [3]:
file = pd.read_csv('customer_data(utf-8).csv',
                   index_col=1,
                   nrows=100000,
                   usecols=USE_COLUMNS,
                   dtype=dtype,
                   parse_dates=PARSE_DATES,
        )

In [4]:
file = file.dropna()

In [5]:
file.shape

(99981, 11)

In [6]:
def get_copurchase_df(df):
    count_size = df.index.unique()
    size_df = df.groupby('交易id').size()
    size_df = size_df[size_df < 2]
    return df.drop(list(size_df.index), axis=0)

In [7]:
data = get_copurchase_df(file)

In [8]:
data.head()

,資料日期與時間,餐別帶,縣市別,店舖代號,主商圈,品號-品名稱,群號-群名稱,單品名稱,銷售數量,銷售單價,交易金額
交易id,,,,,,,,,,,
01637920171201184556000112534201,2017-12-01 18:45:56,晚餐時間帶,台中市,11148,住宅型,72-家庭雜貨,722-家雜用品,（新）銷售用購物袋１８號袋,1,1.0,1.0
01614720171201163042000114405402,2017-12-01 16:30:42,下午茶時間帶,台中市,10697,住宅型,29-冷藏飲料,296-冷藏奶茶,飲冰室茶集綠奶茶,1,25.0,25.0
01384020171201130558000119295301,2017-12-01 13:05:58,午餐時間帶,彰化縣,13840,文教型,12-調理麵,122-熱食麵２配,沙茶牛肉炒麵,1,69.0,69.0
01475120171201153506000111697202,2017-12-01 15:35:06,下午茶時間帶,新竹市,4151,工業型,29-冷藏飲料,291-冷藏茶飲料,冷泡茶台茶１２號紅茶,1,25.0,21.0
01564720171201180924000113751602,2017-12-01 18:09:24,晚餐時間帶,台中市,3860,住宅型,34-健康飲料,343-水,多喝水２Ｌ,2,35.0,49.0


In [9]:
def aggregate(df):
    dic = {}
    for index, row in df.iterrows():
        if index not in dic:
            dic[index] = []
        dic[index].append({
            'name': row['單品名稱'],
            'amount': row['銷售單價']
        })
    dic = { index : value for index, value in dic.items() if len(value) > 1}
    return dic

In [10]:
purchase_list = aggregate(data)

In [11]:
len(purchase_list)

23288

# Network Analysis

In [12]:
from itertools import filterfalse
def convert(purchase_list, support):
    def find_edges_in_list(itemsets):
        from itertools import combinations
        result = []
        return combinations(itemsets, 2)
    result = {}
    nodes = set()
    for key, itemsets in purchase_list.items():
        if len(itemsets) > 1:
            for edge_dict_tuple in find_edges_in_list(itemsets):
                edge = tuple([dic['name'] for dic in edge_dict_tuple])
                weight = sum([dic['amount'] for dic in edge_dict_tuple])
                if edge in result:
                    result[edge]['count'] += 1
                else:
                    result[edge] = {}
                    result[edge]['count'] = 1
                    result[edge]['weight'] = weight
    for key in list(result.keys()):
        if result[key]['count'] < support:
            del result[key]
    for items in result.keys():
        for item in items:
            if item not in nodes:
                nodes.add(item)
    return (nodes, result)

In [13]:
nodes, edges_dict = convert(purchase_list, 8)

In [14]:
print('Node number: {}\nEdge number: {}'.format(len(nodes), len(edges_dict)))

Node number: 194
Edge number: 453


In [35]:
import igraph

In [44]:
g = igraph.Graph()

In [45]:
for node in nodes:
    g.add_vertex(node)

In [46]:
len(g.vs)

194

In [47]:
for edge, attrs in edges_dict.items():
    weight = attrs['weight'] * attrs['count'] if attrs['weight'] > 0 else 1
    g.add_edge(edge[0], edge[1], weight=weight)

In [50]:
g.simplify(combine_edges={ "weight": "sum" })
g.to_undirected(mode='mutual', combine_edges={ "weight": "sum" })

In [51]:
communities = g.community_fastgreedy('weight')

In [55]:
cluster = communities.as_clustering()

In [56]:
len(cluster)

36

In [57]:
for comm in cluster:
    comm_name = [ g.vs[index]['name'] for index in comm]
    print(" ".join(comm_name))
    print("==================")

優格軟糖（Ｐｅａｃｈ） 潤喉糖－蜂蜜檸檬 超涼口香糖（袋） 優格軟糖（Ｏｒａｎｇｅ） 奶香綠茶３３０ＭＬ ＡＷ極酷嗆涼紫冰野莓口香糖 代收折價卷
雲絲頓紅１０毫克香煙 七星１０毫克硬盒香煙 （新）４５號銷售用購物袋 麥香錫蘭奶茶 伯朗咖啡 純喫茶紅茶 冷泡茶冷萃綠茶無糖 飲冰室茶集紅奶茶 紫米紅豆湯圓 咖啡廣場奶香特調咖啡 麥香阿薩姆奶茶 冷泡茶冰釀烏龍 金牌台啤罐裝（６入） 天然水２．２Ｌ 金蘋果調味乳蘋果風味 麥香紅茶ＴＰ３００ 奶油雞排歐姆蛋燴飯 義美奶茶 鹼性離子水 日式蒜香燒豚飯 金牌台灣啤酒５００ＭＬ 天然水 台鹽海洋鹼性離子水 藍山咖啡 泡沫紅茶ＴＰ３００ （新）銷售用購物袋１８號袋 麥香綠茶ＴＰ３００ 純喫茶無糖綠茶６５０ｍｌ 麥香奶茶ＴＰ３００ 飲冰室茶集綠奶茶 純喫茶綠茶６５０ｍｌ 泡沫綠茶ＴＰ３００ 原萃日式綠茶 蔥爆牛肉燴飯 金牌台灣啤酒 茶裏王日式綠茶 全家克林姆麵包 統一大布丁（雞蛋口味） 可口可樂ＰＥＴ 衛生冰
欣中天然氣 健保費代收 花旗信用卡 勞保費代收 國泰世華卡 勞工退休金 台中二段停 遠傳電信 地方查核稅款 台灣電力 欣林瓦斯費 代收手續費４ 台灣自來水 玉山信用卡 聯邦信用卡 台新信用卡 台灣大哥大 國民年金代 中華電信 中信有線代
鮪魚飯糰 簡單點無加糖優酪乳 統一陽光無糖高纖豆漿 義美古早傳統無糖豆奶 全家熱狗麵包 商店街取件 熱拿鐵小杯 鮮奶茶 茶葉蛋（銷售用） 特濃黑可可 蕃薯（２５元） 養樂多 林鳳營全脂鮮乳 光泉無加糖鮮豆漿 肉鬆起酥麵包 蕃薯（２０元） 特濃咖啡拿鐵 香蕉單入 肉鬆飯糰 爆濃起司熱狗 蕃薯（１５元） 光泉米漿 頂級鮮奶優格－莓果穀物脆片 簡單點原味優酪乳 經典原味熱狗 蕃薯（３０元） 雪花蛋糕 ＬＣＡ活菌原味發酵乳 蝦皮取件Ｃ 黑胡椒熱狗
熱拿鐵大杯 冰拿鐵大杯
台鐵手續費 台鐵取票
ｅＴａｇ繳費 通行繳費 代收手續費５
黃金魚豆腐 關東煮本舖拉麵 蟹肉糰子 千層玉子燒 野菜多多魚餅 杏鮑菇 黃金厚切魚板 鮮香菇 王子麵 白玉蘿蔔 讚岐烏龍麵 究極味付蛋 日式黑輪 關東煮本舖冬粉 手工高麗菜捲 旗魚黑輪 筊白筍 特級花枝丸 海鮮魚卵棒
紅標料理米酒 空瓶回收（銷售用）
店到店雅虎拍賣手續費 雅虎拍賣寄件
商店街寄件Ｗ 雅虎線上寄件 店到店雅虎拍賣線上手續費 露天寄件Ｗ 蝦皮寄件Ｗ
蘋果日報 聯合

In [58]:
cluster.modularity

0.821130910093025

In [59]:
items = []
for index, value in enumerate(g.betweenness(weights='weight')):
    if value > 0:
        items.append({ 'name': g.vs[index]['name'], 'betweeness': value })
items.sort(key=lambda x: x['betweeness'], reverse=True)

In [60]:
items[0:10]

[{'name': '（新）銷售用購物袋１８號袋', 'betweeness': 2790.0},
 {'name': '茶葉蛋（銷售用）', 'betweeness': 2564.0},
 {'name': '麥香奶茶ＴＰ３００', 'betweeness': 2011.0},
 {'name': '促銷券０６', 'betweeness': 726.0},
 {'name': '代收折價卷', 'betweeness': 506.0},
 {'name': '熱拿鐵中杯', 'betweeness': 419.0},
 {'name': '經典原味熱狗', 'betweeness': 255.0},
 {'name': '肉鬆飯糰', 'betweeness': 248.0},
 {'name': '鮪魚飯糰', 'betweeness': 173.0},
 {'name': '黑胡椒熱狗', 'betweeness': 172.0}]

In [71]:
g.degree()

[2,
 1,
 1,
 1,
 1,
 5,
 2,
 1,
 1,
 1,
 1,
 1,
 13,
 1,
 2,
 1,
 13,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 7,
 3,
 1,
 3,
 1,
 12,
 1,
 1,
 1,
 1,
 1,
 15,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 3,
 1,
 2,
 11,
 1,
 1,
 9,
 1,
 4,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 25,
 12,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 2,
 1,
 2,
 1,
 4,
 1,
 1,
 16,
 4,
 3,
 1,
 3,
 1,
 1,
 6,
 1,
 2,
 1,
 1,
 2,
 11,
 1,
 12,
 1,
 13,
 1,
 3,
 8,
 1,
 1,
 4,
 1,
 1,
 13,
 1,
 35,
 1,
 1,
 1,
 2,
 7,
 4,
 1,
 1,
 5,
 3,
 1,
 1,
 5,
 2,
 1,
 2,
 1,
 2,
 2,
 3,
 2,
 1,
 2,
 1,
 1,
 1,
 3,
 1,
 12,
 1,
 5,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 8,
 7,
 1,
 1,
 1,
 1,
 1,
 11,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 14,
 2,
 5,
 14,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 14,
 4,
 1,
 1,
 1,
 1,
 4,
 2,
 2]

In [75]:
for index, vertex in enumerate(g.vs):
    vertex.update_attributes({ 'community': cluster.membership[index], 'id': index })

In [87]:
def normalizer(max_degree):
    max_value = max_degree
    min_value = 1
    def normalize(value):
        return (value - min_value) / max_value + 1
    return normalize

In [94]:
import json
def to_json(graph, cluster):
    for index, vertex in enumerate(graph.vs):
        vertex.update_attributes({ 'community': cluster.membership[index], 'id': index })
    norm= normalizer(graph.maxdegree())
    nodes = []
    edges = []
    for edge in graph.es:
        edge_attr = {}
        edge_attr['from'], edge_attr['to'] = edge.tuple
        edge_attr['weight'] = edge['weight']
        edges.append(edge_attr)
    for node in graph.vs:
        node_attr = {}
        node_attr = { key: node[key] for key in node.attributes()}
        node_attr['degree'] = node.degree()
        nodes.append(node_attr)
    return json.dumps({
        'nodes': nodes,
        'edges': edges,
    }, indent=4)

In [95]:
data = to_json(g, cluster)

In [96]:
with open('data.json', 'w', encoding='utf-8') as file:
    file.write(data)